In [48]:
from copy import copy

class Cell:
    
    def __init__(self, pattern='.#./..#/###'):
        self.pattern = pattern
        rows = self.pattern.split('/')
        self.size = len(rows)
        self.kernel = [
            [1 if _ == '#' else 0 for _ in s]
            for s in rows
            ]
        
    def __str__(self):
        buff = []
        for row in range(self.size):
            for col in range(self.size):
                if self.kernel[row][col] == 1:
                    buff.append('#')
                else:
                    buff.append('.')
            buff.append('\n')
        return ''.join(buff)
    
def expand(cell, rule=''):
    if cell.size == 2:  # Multiplo de dos
        new_size = (cell.size // 2) * 3
    else:
        new_size = (cell.size // 3) * 4
    new_pattern = cell.pattern + '.......'
    return Cell(new_pattern)
        
c = Cell()
print(c)

.#.
..#
###



In [49]:
c2 = expand(c)
print(c)

IndexError: list index out of range

In [28]:
class MapOfMaps:
    
    def __init__(self):
        self.size = 3
        self.kernel = [
            [0,1,0],
            [0,0,1],
            [1,1,1],
            ]
        
    def __str__(self):
        buff = []
        for row in range(self.size):
            for col in range(self.size):
                if self.kernel[row][col] == 1:
                    buff.append('#')
                else:
                    buff.append('.')
            buff.append('\n')
        return ''.join(buff)
    
    def expand(self):
        if self.size % 2 == 0:  # Multiplo de dos
            self.size = (self.size // 2) * 3
        else:
            self.size = (self.size // 3) * 4
        
    

In [29]:
mm = MapOfMaps()

In [30]:
print(mm)

.#.
..#
###



In [31]:
for i in range(10):
    print(i, mm.size, end=' -> ')
    mm.expand()
    print(mm.size)

0 3 -> 4
1 4 -> 6
2 6 -> 9
3 9 -> 12
4 12 -> 18
5 18 -> 27
6 27 -> 36
7 36 -> 54
8 54 -> 81
9 81 -> 108
